# Latent Dirichlet Allocation using TensorFlow (Batch VB)

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.eager as tfe

tfe.enable_eager_execution()


### Latent Dirichlet Allocation model:

 - $\theta_{d=1,...,M} \sim \mathrm{Dir}_K(\alpha)$
 - $\beta_{k=1,...,K} \sim \mathrm{Dir}_V(\eta)$
 - $z_{d=1,...,M,i=1,...,N_d} \sim \mathrm{Multinomial}_{ \ K}(\theta_d)$
 - $w_{d=1,...,M,i=1,...,N_d} \sim \mathrm{Multinomial}_{ \ V}(\beta_{z_{di}})$



where

 - $V$ is a vocabulary size
 - $K$ is a number of topics
 - $\eta$ is the parameter of the Dirichlet prior on the per-topic word distribution (known and symetric)
 - $\alpha$ is the parameter of the Dirichlet prior on the per-document topic distributions (known and symetric)
 - $\theta_d$ is the topic distribution for document d
 - $\beta_k$ is the word distribution for topic k
 - $z_{di}$ is the topic for the i-th word in document d
 - $w_{di}$ is a specific word from d-th document belonging to $V$

In plate notation:

![hustlin_erd](LDA_plate_notation.png)

### Posterior distributions:

We assume full factorial design:


$$q(z, \beta, \theta) = \prod_d \prod_i q(z_{di}) \times \prod_d q(\theta_d) \times \prod_k q(\beta_k)$$

where

- $q(z_{di}) = p(z_{di}|\phi) = \mathrm{Multinomial}_{ \ K}(\phi_{dw_{di}}) $
- $q(\theta_d) = p(\theta_d|\gamma) = \mathrm{Dir}_{K}(\gamma_{d}) $
- $q(\beta_k) = p(\beta_k|\lambda) = \mathrm{Dir}_{V}(\lambda_{k}) $

### Parameters Tensors:

*Vocabulary to topic membership:*

$$ \phi = \phi_{d=1,...,D, \ v=1,...,V, \ k=1,...,K} \in \mathbb{R}_{+}^{D \times V \times K}$$
$ \forall_{d \in 1,...,D} \ \phi_d $ is a *stochastic matrix*.

*Distribution of topics:*
$$ \gamma = \gamma_{d=1,...,D, \ k=1,...,K} \in \mathbb{R}_{+}^{D \times K}$$

*Within topic vocabulary profile:*

$$ \lambda = \lambda_{k=1,...,K \ v=1,...,V} \in \mathbb{R}_{+}^{K \times V} $$

### ELBO:

From the definition of the **E**vidence **L**ower **Bo**und:

$$ \mathcal{L}(w, \phi, \gamma, \lambda) := \mathbb{E}_{z, \theta, \beta}\left[\log p(w, z, \theta, \beta| \alpha, \eta) \right] - \mathbb{E}_{z, \theta, \beta}\left[\log q(z, \theta, \beta) \right]$$

Using the probability factorization of the LDA model

$$ \log p(w, z, \theta, \beta| \alpha, \eta) = \log p(w|z, \beta) p(z|\theta) p(\theta | \alpha) p(\beta | \eta) = $$

$$ = \log \left( \prod_{k=1}^K p(\beta_k|\eta) \times \prod_{d=1}^D p(\theta_d|\alpha) \times \prod_{d=1}^{D} \prod_{i=1}^{N_d} p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) = $$

$$ =  \sum_{k=1}^K \log p(\beta_k|\eta) +  \sum_{d=1}^D \log p(\theta_d|\alpha) + \sum_{d=1}^{D} \sum_{i=1}^{N_d} \log p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) = $$

$$ =  \sum_{k=1}^K \log p(\beta_k|\eta) +  \sum_{d=1}^D \left( \log p(\theta_d|\alpha) + \sum_{i=1}^{N_d} \log p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) = $$

$$ =  \sum_{d=1}^D \left( \log p(\theta_d|\alpha) + \sum_{i=1}^{N_d} \log \left( p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) + \frac{1}{D} \sum_{k=1}^K \log p(\beta_k|\eta) \right) $$


Taking the expectations with respect to the posterior distribution results in:

$$ \mathbb{E}_{\theta} \left[ \log p(\theta_d|\alpha) \right] = C_1(\alpha) + \sum_{k=1}^K (\alpha_k - 1) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] $$

$$ \mathbb{E}_{z, \beta} \left[ \log p(w_{di}|\beta_{z_{di}}) \right] =  \mathbb{E}_{z, \beta} \left[ \log \beta_{z_{di} i} \right] = \sum_{k=1}^{K} \phi_{dw_{di}k} \mathbb{E}_{\beta} \left[ \log \beta_{k i} \right] $$

$$ \mathbb{E}_{z, \theta} \left[ \log p(z_{di}|\theta_d) \right] =  \sum_{k=1}^{K} \phi_{dw_{di}k} \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] $$

$$ \mathbb{E}_{\beta} \left[ \log p(\beta_{k}|\eta) \right] = C_2(\eta) + \sum_{v=1}^{V} (\eta_v - 1) \mathbb{E}_{\beta} \left[ \log \beta_{kv} \right] $$

For the entropy of the posterior distribution we have:

$$ \log\left(\prod_{d=1}^{D} \prod_{i=1}^{N_{d}} q(z_{di}) \times \prod_{d=1}^D q(\theta_d) \times \prod_{k=1}^{K} q(\beta_k)\right) =  $$

$$ = \sum_{d=1}^{D} \left( \sum_{i=1}^{N_{d}} \log q(z_{di}) + \log q(\theta_d) + \frac{1}{D} \sum_{k=1}^{K} \log q(\beta_k)\right) $$

and expectations with respect to the posterior distribution:

$$ \mathbb{E}_{z} \left[ \log q(z_{di})  \right] = \mathbb{E}_{z_{di}} \left[ \mathbb{1}_{[z_{di}=k]} \log \phi_{dw_{di}k}  \right] = \sum_{k=1}^{K} \phi_{dw_{di}k} \log \phi_{dw_{di}k} $$

$$ \mathbb{E}_{\theta} \left[ \log q(\theta_d)  \right] = \mathbb{E}_{\theta} \left[ \log \frac{1}{B(\gamma_d)} \prod_{k=1}^{K} \theta_{dk}^{\gamma_{dk} - 1} \right] = -\log B(\gamma_d) + \sum_{k=1}^K (\gamma_{dk} - 1)\mathbb{E}_{\theta} \left[ \log \theta_{dk} \right]$$

$$ \mathbb{E}_{\beta}\left[ \log q(\beta_k) \right] = -\log B(\lambda_k) + \sum_{v=1}^V (\lambda_{kv} - 1)\mathbb{E}_{\beta} \left[ \log \beta_{kv} \right]$$

*We should then be able to collapse ELBO to the formula based on counts of words rather than words from documents. This will show that the LDA model is fixed size method.* 

*"When training an LDA model, you start with a collection of documents and each of these is represented by a fixed-length vector (bag-of-words). LDA is a general Machine Learning (ML) technique, which means that it can also be used for other unsupervised ML problems where the input is a collection of fixed-length vectors and the goal is to explore the structure of this data."* (Data Camp)

Grouping terms by corresponding parameters we get:

#### Document vocabulary membership

$$ \mathcal{L}_{[\phi_{dv}]} = \sum_{d=1}^{D} \sum_{i=1}^{N_d} \sum_{k=1}^K \phi_{dw_{di}k} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k i} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dw_{di}k}  \right) = $$

$$ = \sum_{d=1}^{D} \sum_{v=1}^{V} \sum_{k=1}^K n_{dv} \phi_{dvk} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk}  \right)$$

where $n_{dv}$ is a number of times word $v$ from vocabulary $V$ was counted in document $d$.

#### Distribution of topics

$$ \mathcal{L}_{[\gamma_{d}]} = \sum_{d=1}^{D} \sum_{k=1}^K \left\{ (\alpha_k - 1) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] + \log B(\gamma_d) - (\gamma_{dk} - 1)\mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] \right\} = $$

$$ = \sum_{d=1}^{D} \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk}) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] + \frac{1}{K} \log B(\gamma_d) \right\}$$

#### Topic vocabulary profile

$$ \frac{1}{D} \sum_{d=1}^{D} \sum_{v=1}^{V} \sum_{k=1}^K \left\{ (\eta_v - \lambda_{kv}) \mathbb{E}_{\beta} \left[ \log \beta_{k} \right] + \frac{1}{V} \log B(\lambda_k) \right\}$$

### Coordinate ascent optimization

Optimizing for every document and vocabulary entry separately we get:

#### Document vocabulary membership:

$$ \mathrm{argmax}_{\phi_{dv} \in \Delta^{K-1}} \mathcal{L}_{[\phi_{dv}]} =  \mathrm{argmax}_{\phi_{dv} \in \Delta^{K-1}} \sum_{k=1}^K n_{dv} \phi_{dvk} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk}  \right) = $$

$$ =\mathrm{argmax}_{\phi_{dv} \in \mathbb{R}_{+}^K} \sum_{k=1}^K n_{dv} \phi_{dvk} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk}  \right) + \xi (\sum_{k=1}^K \phi_{dvk} - 1) = \Psi(\phi_{dv}, \xi) $$

$$ \frac{\partial}{\partial \phi_{dvk}} \Psi(\phi_{dv}, \xi) = n_{dv} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk} - 1 \right) + \xi $$

$$ \frac{\partial}{\partial \xi} \Psi(\phi_{dv}, \xi) = \sum_{k=1}^K \phi_{dvk} - 1 $$

$$ \frac{\partial}{\partial \phi_{dvk}} \Psi(\phi_{dv}, \xi) = 0 \iff n_{dv} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk} - 1 \right) + \xi = 0 \iff $$

$$ \iff \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk} - 1 + \xi' = 0 \iff $$

$$ \iff \log \phi_{dvk} = \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - 1 + \xi'  \iff $$

$$ \iff \phi_{dvk} \propto \exp \left\{ \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] \right\} $$

#### Doucuments' distribution of topics

Since

$$ \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] = \psi(\gamma_{dk}) - \psi(\sum_{k=1}^K\gamma_{dk})$$

and

$$ \log B(\gamma_d) = \log \frac{\prod_{k=1}^{K}\Gamma(\gamma_{dk})}{\Gamma ({\sum_{k=1}^K \gamma_{dk}})} = \sum_{k=1}^{K}\log \Gamma (\gamma_{dk}) - \log \Gamma ({\sum_{k=1}^K \gamma_{dk}})$$

$$ \mathrm{argmax}_{\gamma_{d} \in \mathbb{R}_{+}^{K}} \mathcal{L}_{[\gamma_{d}]} = \mathrm{argmax}_{\gamma_{d} \in \mathbb{R}_{+}^{K}} \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk}) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] \right\} + \log B(\gamma_d) = $$

$$ = \mathrm{argmax}_{\gamma_{d} \in \mathbb{R}_{+}^{K}} \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk}) (\psi(\gamma_{dk}) - \psi(\sum_{k=1}^K\gamma_{dk})) \right\} + \sum_{k=1}^{K}\log \Gamma (\gamma_{dk}) - \log \Gamma ({\sum_{k=1}^K \gamma_{dk}}) = \Psi(\gamma_d)$$

$$ \frac{\partial}{\partial \gamma_{dj}} \Psi(\gamma_d) = -\psi(\gamma_{dj}) + \psi({\sum_k \gamma_{dk})} + \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk})*(\psi'(\gamma_{dj})*1_{k=j} - \psi'(\sum_{k=1}^K\gamma_{dk})) \right\} + \psi(\gamma_{dj}) - \psi(\sum_k \gamma_{dk}) = $$

$$ = \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk})*(\psi'(\gamma_{dj})*1_{k=j} - \psi'(\sum_{k=1}^K\gamma_{dk})) \right\} $$

Since *trigamma* function has no real roots it follows that:

$$ \frac{\partial}{\partial \gamma_{dj}} \Psi(\gamma_d) = 0 \iff \forall_{k} \ \ \  \alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk} = 0 \iff $$

$$ \iff   \gamma_{dk} = \alpha_k  + \sum_{v=1}^V n_{dv} \phi_{dvk} $$

#### Topic vocabulary profile (M-Step)

Exact derivation as for the $\gamma_d$ gives an update equation for $\lambda_k$

$$ \lambda_{kv} = \eta_v + \sum_{d=1}^D n_{dv}\phi_{dvk} $$

### Final algorithm

 - initialize $\lambda$ and $\gamma$
 - while improvement $ \mathcal{L}(w, \phi, \gamma, \lambda) > 1e-6$ 
     - for $d = 1,...,D$
         - $\phi_{dvk} \propto \exp \left\{ \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] \right\}$
         - $\gamma_{dk} = \alpha  + \sum_{v=1}^V n_{dv} \phi_{dvk}$
     - $\lambda_{kv} = \eta_v + \sum_{d=1}^D n_{dv}\phi_{dvk}$

### Generate data

In [232]:
# set data dimensions
K = 2
V = 4
D = 5
N = 100

# set the seed
np.random.seed(2014)

# beta prior parameters
eta = np.ones(V)

# beta profiles
beta = np.random.dirichlet(alpha=eta, size=K)

# theta prior parameters
alpha = np.ones(K)

# document's prior topic allocation
theta = np.random.dirichlet(alpha=alpha, size=D)

# word's topic membership
z = [np.random.choice(K, size=N, replace=True, p=theta[d, :]) for d in range(D)]
z = np.vstack(z)

# actual words and counts
w = [np.array([np.random.choice(V, size=1, p=beta[k,:])[0] for k in z[d, :]]  + list(range(V))) for d in range(D)]
nw = [np.unique(w[d], return_counts=True)[1] for d in range(D)]
nw = np.vstack(nw)
w = np.vstack(w)

nw = tf.convert_to_tensor(nw, dtype=tf.float32)
nw

<tf.Tensor: id=1448625, shape=(5, 4), dtype=float32, numpy=
array([[19., 31., 33., 21.],
       [ 4., 14., 15., 71.],
       [ 4., 10., 10., 80.],
       [10., 20., 23., 51.],
       [ 7., 15., 11., 71.]], dtype=float32)>

### Tensorflow model

Few notes:
- there is a tape now for gradients like in pyTorch
- we will only use tensorflow for fast tensor operations

In [233]:
# initialize LDA parameters
def initialize_parameters(K, V, D, alpha=1e-2, eta=1e-2, seed=2014):
    """
    Initialize parameters of LDA model returning adequate Tensors.

    args:
    
        K (int): number of LDA components 
        V (int): vocabulary size
        D (int): number of documents
        alpha (float): hyperparameter for theta prior
        eta (float): hyperparameter for beta prior
       
       
    returns:
    
        eta: [1 x V] tensor with prior parameters (alpha) for beta
        lambda: k x [1, V] tensors with posterior word distribution per class
        phi: [D, V, K] tensor with vocabulary membership per document
        gamma: k x [D, 1] tensors
        
    """
    
    eta = tf.ones((1, V)) * alpha
    lambda_ = [tf.abs(tf.random_normal(shape=(1, V), seed=seed)) for k in range(K)]
    
    phi = tf.random_normal(shape=(D, V, K), seed=seed)
    phi = tf.nn.softmax(phi, axis=2)
    
    gamma = [tf.abs(tf.random_normal(shape=(1, V), seed=seed)) for k in range(K)]
    
    return eta, lambda_, phi

eta, lambda_, phi = initialize_parameters(K, V, D)


In [234]:
lambda_

[<tf.Tensor: id=1448634, shape=(1, 4), dtype=float32, numpy=array([[0.02746824, 0.6522103 , 0.9192426 , 1.4331781 ]], dtype=float32)>,
 <tf.Tensor: id=1448639, shape=(1, 4), dtype=float32, numpy=array([[2.0289245 , 0.1510453 , 0.96964526, 0.11797091]], dtype=float32)>]

In [230]:
# update for lambda
def update_lambda(lambda_, eta, phi):
    
    K = len(lambda_)
    for k in range(K):
        lambda_[k] = tf.add(tf.reduce_sum(tf.multiply(phi[:,:,k], nw), axis=0, keepdims=True), eta)
    return lambda_ 

In [ ]:
def e_log_theta()



In [228]:
K = 100
V = 100000

for i in range(10):

    b = tf.random_normal((K, V))
    b = tf.transpose(b)
    b = tf.expand_dims(input=b, axis=0)
    b = tf.tile(b, multiples=[D, 1, 1])
    print(tf.reduce_sum(b))

# b_dvk = 
# tf.tile(tf.expand_dims(tf.transpose(b)), multiples=[D, 1, 1])
# a = tf.random_normal((D, K))
# 


tf.Tensor(-7050.6655, shape=(), dtype=float32)
tf.Tensor(8332.171, shape=(), dtype=float32)
tf.Tensor(-782.49097, shape=(), dtype=float32)
tf.Tensor(-18377.848, shape=(), dtype=float32)
tf.Tensor(4282.1626, shape=(), dtype=float32)
tf.Tensor(2693.7761, shape=(), dtype=float32)
tf.Tensor(26654.258, shape=(), dtype=float32)
tf.Tensor(6411.775, shape=(), dtype=float32)
tf.Tensor(-11554.484, shape=(), dtype=float32)
tf.Tensor(19046.424, shape=(), dtype=float32)


In [235]:
?tf.stack

Signature: tf.stack(values, axis=0, name='stack')
Docstring:
Stacks a list of rank-`R` tensors into one rank-`(R+1)` tensor.

Packs the list of tensors in `values` into a tensor with rank one higher than
each tensor in `values`, by packing them along the `axis` dimension.
Given a list of length `N` of tensors of shape `(A, B, C)`;

if `axis == 0` then the `output` tensor will have the shape `(N, A, B, C)`.
if `axis == 1` then the `output` tensor will have the shape `(A, N, B, C)`.
Etc.

For example:

```python
x = tf.constant([1, 4])
y = tf.constant([2, 5])
z = tf.constant([3, 6])
tf.stack([x, y, z])  # [[1, 4], [2, 5], [3, 6]] (Pack along first dim.)
tf.stack([x, y, z], axis=1)  # [[1, 2, 3], [4, 5, 6]]
```

This is the opposite of unstack.  The numpy equivalent is

```python
tf.stack([x, y, z]) = np.stack([x, y, z])
```

Args:
  values: A list of `Tensor` objects with the same shape and type.
  axis: An `int`. The axis to stack along. Defaults to the first dimension.
    Negative val